In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [36]:
import logging
from pathlib import Path
import textwrap
import copy
from PIL import Image, ImageDraw, ImageFont
from waveshare_epd import epd2in7b

In [16]:
# CONSTANTS 
FONT = './fonts/Open_Sans/OpenSans-Regular.ttf'
CHARDIST = dist = {
'A': 0.0796394951934866,
'B': 0.0480421724743304,
'C': 0.0650193941562118,
'D': 0.0367478368492653,
'E': 0.0392455626459735,
'F': 0.0285607049987683,
'G': 0.0264235633824497,
'H': 0.0350469680738427,
'I': 0.063304067996198,
'J': 0.0223114296397362,
'K': 0.0132044112598647,
'L': 0.0303276241783032,
'M': 0.0735552041056704,
'N': 0.058228959048466,
'O': 0.0299636382603628,
'P': 0.0408886018830318,
'Q': 0.00330507150877549,
'R': 0.0415148050705166,
'S': 0.0864526085515713,
'T': 0.092261358897769,
'U': 0.0162965906935831,
'V': 0.00880284634720004,
'W': 0.0303874380635722,
'X': 0.00214819726335884,
'Y': 0.026731137152672,
'Z': 0.0015903123050202,
'a': 0.0853444241209682,
'b': 0.014043443886782,
'c': 0.0317851933335035,
'd': 0.0384231410874873,
'e': 0.125522566035832,
'f': 0.0210277287958113,
'g': 0.019565625337748,
'h': 0.0479248841551587,
'i': 0.0734040206369666,
'j': 0.00106775804890564,
'k': 0.00747099878278566,
'l': 0.0413955994606343,
'm': 0.0237913407247777,
'n': 0.0735371823360625,
'o': 0.0766780830435463,
'p': 0.0203573642991815,
'q': 0.000879113659646998,
'r': 0.0670907487656561,
's': 0.0678732298030442,
't': 0.0892991141869109,
'u': 0.0261576563826317,
'v': 0.0105934322827605,
'w': 0.0164673662068651,
'x': 0.00200361905383887,
'y': 0.0172194144536526,
'z': 0.00107695111884201,
}

In [17]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [89]:
class textblock:
    def __init__(self, epd=None, font=FONT, size=24, maxRows=3, charPerRow=24):

        self.height = epd2in7b.EPD().height
        self.width = epd2in7b.EPD().width
        self.font = self.setFont(fontPath=font, size=size)
        self.charPerRow = charPerRow
        self.maxRows = maxRows
    
    def setFont(self, fontPath, size):
        font = ImageFont.truetype(str(Path(fontPath).absolute()), size)
        return(font)
    
    
    @property
    def charPerRow(self):
        return self.__charPerRow
    
    @charPerRow.setter
    def charPerRow(self, charPerRow):
        s = ''
        n = 1000
        testFont = self.font
        # create a string of characters containing the letter distribution
        for char in CHARDIST:
            s = s+(char*int(dist[char]*n))
        sLenPX = testFont.getsize(s)[0] # string length in Pixles
        avgCharLen = sLenPX/len(s)
        charPerRow = round(self.height/avgCharLen)        
        self.__charPerRow = charPerRow

    def wrapText(self, text='', charPerRow=None, maxRows=None):
        if not charPerRow:
            charPerRow = self.charPerRow
        if not maxRows:
            maxRows = self.maxRows
        try:
            wrapper = textwrap.wrap(text, width=charPerRow)
        except AttributeError as e:
            raise ValueError(f'Attribute Error: {text} is not a string; it appears to be {type(text)}')
        myText = "\n".join(wrapper[0:maxRows])
        return(myText)
    
    def textImg(self, text):
        black = Image.new('1', (epd.height, epd.width), 255)
        other = Image.new('1', (epd.height, epd.width), 255)
        drawBlack = ImageDraw.Draw(black)
        drawOther = ImageDraw.Draw(other)
        myText = self.wrapText(text)
        drawBlack.text((0, 0), myText, font=self.font, fill=0)
        
        return(black, other)
    
    

In [100]:
track = textblock(maxRows=3, size=36)
track.wrapText('Old education him departure any arranging one prevailed. Their end whole might began her. Behaved the comfort another fifteen eat. Partiality had his themselves ask pianoforte increasing discovered. So mr delay at since place whole above miles. He to observe conduct at detract because. Way ham unwilling not breakfast furniture explained perpetual. Or mr surrounded conviction so astonished literature. Songs to an blush woman be sorry young. We certain as removal attempt.')

'Old education\nhim departure\nany arranging'

In [102]:
textImg = track.textImg('Old education him departure any arranging one prevailed. Their end whole might began her. Behaved the comfort another fifteen eat. Partiality had his themselves ask pianoforte increasing discovered. So mr delay at since place whole above miles. He to observe conduct at detract because. Way ham unwilling not breakfast furniture explained perpetual. Or mr surrounded conviction so astonished literature. Songs to an blush woman be sorry young. We certain as removal attempt.')

In [92]:
epd = epd2in7b.EPD()

In [79]:
epd.init()

DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release


0

In [103]:
textImg[0].save('blk2.png')
f = textImg[0]
b = textImg[1]

In [104]:
epd.display(epd.getbuffer(textImg[1]), epd.getbuffer(textImg[0]))

DEBUG:root:Horizontal
DEBUG:root:Horizontal
DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release
